In [2]:
# coding=utf-8
import re
import pymorphy2
import pandas as pd
import json
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
from matplotlib import rc
import matplotlib

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

In [3]:
queries = [  u'("подскажите" | "не подскажите" |  "не подскажете" | "посоветуйте" | "порекомендуйте" | "предложите" | "можете подсказать" | "посоветуй" | "подскажи" | "порекомендуй")',
             u'("ищу" | "разыскиваю" | "нужен" | "нужна" | "найдись" | "требуется" | "разыскивается" | "необходим")',
             u'("необходимо найти" | "надо найти" | "хочу найти" | "хотим найти" | "помогите найти" | "хотелось бы найти" | "надо сделать" | "хочу сделать" | "хотим сделать" | "помогите сделать" | "хотелось бы сделать" | "необходимо создать" | "надо создать" | "хочу создать" | "хотим создать" | "помогите создать" | "хотелось бы создать")',
             u'("в поисках"  | "в поиске")',
             u'("интересует" | "интересуемся")',
             u'("хочу" | "хотим" | "хотелось бы" | "планирую " | "планируем" | "собираюсь" | "собираемся" | "собралась" | "собрался")',
             u'("думаю" | "думаем")',
             u'("выбираю" | "выбираем" | "присматриваю" | "присматриваем"  | "подбираю" | "подбираем" | "подыскиваю" | "подыскиваем")',
             u'(интересует" | "сориентируйте" | "хочу" | "скажите" | "напишите" | "какая " | " сколько ")',
             u'("дайте" | "подбросьте"  | "подкиньте" | "напишите" | "киньте | "скажите")',
             u'("нужна консультация" | "необходима консультация" | "нужна помощь" | "нужна консультация" | "нужнен совет")',
             u'("поделитесь советами" | "поделитесь советом" | "поделитесь мнением" | "поделитесь мнениями" | "поделитесь опытом" | "поделитесь отзывами" | "поделитесь отзывом" | "поделитесь информацией" | "поделитесь инфой")',
             u'("помогите советом" | "помогите опытом" | "помогите разобраться")',
             u'("куда обратиться" | "куда обратится" | "где глянуть" | "где посмотреть")',
             u'("здесь есть" | "есть здесь" | "есть ли" | "нужно ли" | "возможно ли" | "есть у кого" | "у кого есть" | "кто здесь" | "кто тут" | "кто может")',
             u'("кто-нибудь")',
             u'("какие есть" | "какие существуют")',
             u'("встречал кто" | "кто встречал" | "кто знает" | "знает кто" | "никто не знает" | "что можно" | "что думаете" | "что лучше")',
             u'("еще вопрос" | "осваиваю" | "такой вопрос" | "проблема такая" | "такая проблема" | "возникла проблема")',
             u'("можно ли" | "как лучше" | "сталкивался кто" | "может кто-то" | "кто-то сталкивался" | "кто-нибудь сталкивался")',
             u'("какой движок" | "призываю знатоков" | "выдало ошибку" | "не могу понять")',
            ]            

In [4]:
def extract_keywords(part):
    search_words = part['text'].split('|')
    exact_keywords = []
    for search_word in search_words:
        quoted_word_match = re.findall('"([^"]*)"', search_word)
        if quoted_word_match:
            exact_keywords.append(quoted_word_match[0])
    if exact_keywords:
        part['exact_keywords'] = exact_keywords
    return part

In [5]:
def register_query(query):
    parts = [{'text': t} for t in query.split("/")]
    parts = [extract_keywords(part) for part in parts]
    return parts

In [6]:
def register_queries(queries):
    return [register_query(query) for query in queries]

In [7]:
ordered_queries = {}
for query in register_queries(queries):
    if len(query) in ordered_queries:
        ordered_queries[len(query)].append({
            'query': query,
        })
    else:
        ordered_queries[len(query)] = [{
            'query': query,
            }]

In [8]:
list_of_queries = register_queries(queries)

In [9]:
def match_cases(message):
    for lists in list_of_queries:
        if any([word in message for word in lists[0]['exact_keywords']]):
            return message

In [14]:
train_set = pd.read_csv("train_set.csv")
channels = ['big_data', 'datasets', 'deep_learning', 'lang_python', 'lang_r', 'mltrainings_beginners', 'nlp', 'sequences_series', 'theory_and_practice', 'visualization']
train_set = train_set[train_set['channel'].isin(channels)]
train_set = train_set[(train_set['timestamp'] > '2017-01-01')] #& (train_set['timestamp'] > '2016-01-01')]

#train_set[1:4]

users = pd.read_json('data/users.json')
users = users.rename(columns={'id': 'user_id'})
train_set = pd.merge(train_set, users[['user_id','real_name']], on='user_id')

In [11]:
def novice_filter(train_set):
    idxs = train_set.apply(lambda row: True if match_cases(str(row["text"])) else False, axis = 1)
    return train_set[idxs]

In [12]:
novices = novice_filter(train_set)
novices.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3235 entries, 29 to 38934
Data columns (total 12 columns):
Unnamed: 0          3235 non-null int64
msg_id              3235 non-null int64
user_id             3235 non-null object
channel             3235 non-null object
timestamp           3235 non-null object
text                3235 non-null object
main_msg            3235 non-null int64
thread_id           3235 non-null int64
thread_timestamp    3235 non-null object
replies_count       3235 non-null int64
reactions           3235 non-null object
real_name           3234 non-null object
dtypes: int64(5), object(7)
memory usage: 328.6+ KB


In [13]:
user_list = novices['real_name']
frequency = {}
for word in user_list:
    count = frequency.get(word,0)
    frequency[word] = count + 1
frequency_list = frequency.keys()
novices_list = pd.DataFrame(list(frequency.items()),
                      columns=['name','count'])

novices_list = novices_list.sort('count', ascending=False)
novices_list = novices_list.to_csv('novices_list_2017.csv', sep='\t', encoding='utf-8')

/home/kateryna/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:10: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
